In [1]:
import tensorflow.compat.v2 as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pathlib
import random
import glob
import os
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
import time
keras = tf.keras
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [2]:
def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32)
    image = image/255.0
    image = 2*image-1
    return image

In [3]:
train_dir = r'C:\Users\24306\Desktop\datasets\kfold\4\train'
test_dir  = r'C:\Users\24306\Desktop\datasets\kfold\4\test'
checkpoint_path = r'C:\Users\24306\Desktop\NASNetMobile\4\NASNetMobile.ckpt'

# 处理训练数据

In [4]:
#打乱顺序
train_root = pathlib.Path(train_dir)
train_image_paths = list(train_root.glob('*/*'))
train_image_paths = [str(path) for path in train_image_paths]
random.shuffle(train_image_paths)
#处理训练标签
train_image_labels = [pathlib.Path(path).parent.name for path in train_image_paths]
train_labels = [[label.split('_')[0]] for label in train_image_labels]
new_train_labels=[]
for n in train_labels:
    new_train_labels.append(float(n[0]));  
train_labels=new_train_labels
print(train_labels)
#处理训练图像
train_path_ds = tf.data.Dataset.from_tensor_slices(train_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_image_ds = train_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
train_label_ds = tf.data.Dataset.from_tensor_slices(train_labels)
train_image_label_ds = tf.data.Dataset.zip((train_image_ds, train_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
train_count = len(train_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
train_data = train_image_label_ds.shuffle(buffer_size=train_count).repeat(-1)
train_data = train_data.batch(BATCH_SIZE)
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(train_data)



[86.0, 84.0, 20.0, 66.0, 16.0, 95.0, 84.0, 96.0, 47.0, 40.0, 47.0, 84.0, 96.0, 37.0, 100.0, 95.0, 47.0, 100.0, 37.0, 10.0, 40.0, 86.0, 96.0, 16.0, 93.0, 47.0, 23.0, 37.0, 100.0, 32.0, 86.0, 45.0, 86.0, 86.0, 66.0, 45.0, 23.0, 96.0, 40.0, 45.0, 93.0, 45.0, 37.0, 35.0, 37.0, 16.0, 47.0, 23.0, 47.0, 95.0, 37.0, 7.0, 13.0, 16.0, 95.0, 13.0, 100.0, 80.0, 10.0, 55.0, 80.0, 10.0, 96.0, 100.0, 95.0, 52.0, 100.0, 93.0, 37.0, 23.0, 93.0, 35.0, 100.0, 66.0, 84.0, 86.0, 10.0, 32.0, 16.0, 32.0, 80.0, 80.0, 37.0, 10.0, 86.0, 80.0, 47.0, 95.0, 35.0, 45.0, 37.0, 100.0, 86.0, 39.0, 84.0, 13.0, 23.0, 47.0, 47.0, 55.0, 4.0, 37.0, 16.0, 47.0, 95.0, 30.0, 40.0, 16.0, 100.0, 40.0, 32.0, 13.0, 40.0, 80.0, 100.0, 13.0, 86.0, 47.0, 40.0, 52.0, 47.0, 7.0, 10.0, 23.0, 96.0, 80.0, 95.0, 84.0, 55.0, 37.0, 47.0, 13.0, 100.0, 16.0, 100.0, 66.0, 39.0, 10.0, 20.0, 10.0, 84.0, 52.0, 80.0, 35.0, 16.0, 66.0, 100.0, 96.0, 100.0, 32.0, 40.0, 10.0, 86.0, 47.0, 37.0, 40.0, 93.0, 13.0, 16.0, 47.0, 20.0, 37.0, 32.0, 86.0, 84.0

# 处理验证数据

In [5]:
#打乱顺序
test_root = pathlib.Path(test_dir)
test_image_paths = list(test_root.glob('*/*'))
test_image_paths = [str(path) for path in test_image_paths]
#random.shuffle(train_image_paths)
#处理训练标签
test_image_labels = [pathlib.Path(path).parent.name for path in test_image_paths]
test_labels = [[label.split('_')[0]] for label in test_image_labels]
new_test_labels=[]
for n in test_labels:
    new_test_labels.append(float(n[0]));  
test_labels=new_test_labels
print(test_labels)
#处理训练图像
test_path_ds = tf.data.Dataset.from_tensor_slices(test_image_paths)
AUTOTUNE = tf.data.experimental.AUTOTUNE
test_image_ds = test_path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
test_label_ds = tf.data.Dataset.from_tensor_slices(test_labels)
test_image_label_ds = tf.data.Dataset.zip((test_image_ds, test_label_ds))
# image_count = len(all_image_paths)

# test_count = int(image_count*0.2)
test_count = len(test_image_paths)
# train_data = image_label_ds.skip(test_count)
# test_data = image_label_ds.take(test_count)

BATCH_SIZE = 32
test_data = test_image_label_ds.shuffle(buffer_size=test_count).repeat(-1)
test_data = test_data.batch(BATCH_SIZE)
test_data = test_data.prefetch(buffer_size=AUTOTUNE)

#test_data = test_data.batch(BATCH_SIZE)
print(test_data)

[10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.

In [6]:
#要改
NASNetMobile = tf.keras.applications.NASNetMobile(weights='imagenet',include_top=False,input_shape=(224, 224, 3))
#要改
NASNetMobile.trainable = True
inputs = tf.keras.Input(shape=(224, 224, 3))
#要改
x = NASNetMobile(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
out_weight = tf.keras.layers.Dense(1,name='out_weight')(x)

model = tf.keras.Model(inputs=inputs,
                       outputs=[out_weight])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 global_average_pooling2d (G  (None, 1056)             0         
 lobalAveragePooling2D)                                          
                                                                 
 out_weight (Dense)          (None, 1)                 1057      
                                                                 
Total params: 4,270,773
Trainable params: 4,234,035
Non-trainable params: 36,738
_________________________________________________________________


In [7]:
print ('start time:',time.asctime( time.localtime(time.time()) ))
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, decay=0.01),loss='mse')

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,save_best_only=True,monitor='val_loss')
history = model.fit(train_data,
                    steps_per_epoch=train_count//BATCH_SIZE,
                    epochs=300,
                    validation_data=test_data,
                    validation_steps=test_count//BATCH_SIZE,
                   callbacks=[cp_callback])
#要改
np.savetxt('NASNetMobile_loss_4.txt',history.history['loss'])
#要改
np.savetxt('NASNetMobile_valloss_4.txt',history.history['val_loss'])
print ('end time:',time.asctime( time.localtime(time.time()) ))

start time: Sun Apr  9 23:41:58 2023
Epoch 1/300
592/592 [==============================] - ETA: 0s - loss: 65.8649

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\4\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\4\NASNetMobile.ckpt\assets


592/592 [==============================] - 190s 272ms/step - loss: 65.8649 - val_loss: 803.0497
Epoch 2/300
592/592 [==============================] - ETA: 0s - loss: 2.1218

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\4\NASNetMobile.ckpt\assets


...

INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\4\NASNetMobile.ckpt\assets


INFO:tensorflow:Assets written to: C:\Users\24306\Desktop\NASNetMobile\4\NASNetMobile.ckpt\assets


592/592 [==============================] - 208s 352ms/step - loss: 0.0390 - val_loss: 0.0628
Epoch 55/300
592/592 [==============================] - 123s 208ms/step - loss: 0.0725 - val_loss: 0.1425
Epoch 56/300
592/592 [==============================] - 121s 204ms/step - loss: 0.0422 - val_loss: 0.0834
Epoch 57/300
592/592 [==============================] - 121s 204ms/step - loss: 0.0403 - val_loss: 0.1194
Epoch 58/300
592/592 [==============================] - 121s 204ms/step - loss: 0.0372 - val_loss: 0.1275
Epoch 59/300
592/592 [==============================] - 121s 204ms/step - loss: 0.0365 - val_loss: 0.1466
Epoch 60/300
592/592 [==============================] - 121s 204ms/step - loss: 0.0379 - val_loss: 0.1238
Epoch 61/300
592/592 [==============================] - 122s 205ms/step - loss: 0.0361 - val_loss: 0.1421
Epoch 62/300
592/592 [==============================] - 121s 205ms/step - loss: 0.0344 - val_loss: 0.1378
Epoch 63/300
592/592 [==============================] - 122

592/592 [==============================] - 127s 215ms/step - loss: 0.0302 - val_loss: 0.1007
Epoch 132/300
592/592 [==============================] - 128s 216ms/step - loss: 0.0242 - val_loss: 0.1172
Epoch 133/300
592/592 [==============================] - 128s 216ms/step - loss: 0.0255 - val_loss: 0.1210
Epoch 134/300
592/592 [==============================] - 129s 217ms/step - loss: 0.0260 - val_loss: 0.1557
Epoch 135/300
592/592 [==============================] - 128s 216ms/step - loss: 0.0246 - val_loss: 0.1373
Epoch 136/300
592/592 [==============================] - 128s 216ms/step - loss: 0.0233 - val_loss: 0.1330
Epoch 137/300
592/592 [==============================] - 128s 217ms/step - loss: 0.0235 - val_loss: 0.1255
Epoch 138/300
592/592 [==============================] - 128s 216ms/step - loss: 0.0337 - val_loss: 0.1377
Epoch 139/300
592/592 [==============================] - 128s 217ms/step - loss: 0.0278 - val_loss: 0.1584
Epoch 140/300
592/592 [============================

592/592 [==============================] - 140s 237ms/step - loss: 0.0177 - val_loss: 0.1105
Epoch 284/300
592/592 [==============================] - 140s 236ms/step - loss: 0.0239 - val_loss: 0.1562
Epoch 285/300
592/592 [==============================] - 140s 237ms/step - loss: 0.0180 - val_loss: 0.1506
Epoch 286/300
592/592 [==============================] - 140s 236ms/step - loss: 0.0185 - val_loss: 0.1469
Epoch 287/300
592/592 [==============================] - 139s 235ms/step - loss: 0.0249 - val_loss: 0.1310
Epoch 288/300
592/592 [==============================] - 140s 236ms/step - loss: 0.0184 - val_loss: 0.1389
Epoch 289/300
592/592 [==============================] - 140s 236ms/step - loss: 0.0382 - val_loss: 0.1238
Epoch 290/300
592/592 [==============================] - 139s 234ms/step - loss: 0.0185 - val_loss: 0.1358
Epoch 291/300
592/592 [==============================] - 140s 236ms/step - loss: 0.0181 - val_loss: 0.1121
Epoch 292/300
592/592 [============================